In [ ]:
!pip install pyspark==2.4.5
!pip install lightning-python

In [11]:
import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars spark-streaming-kafka-0-8-assembly_2.11-2.4.5.jar pyspark-shell' 
# Downloaded from: https://mvnrepository.com/artifact/org.apache.spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.5
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from time import sleep
from lightning import Lightning
import json
import numpy as np
import requests

#### Init Lightning

In [ ]:
# Init LIGHTNING-VIZ server
lgn = Lightning(host="http://localhost:3000")
lgn.create_session("Rojo")

series = np.random.randn(3,1)
viz = lgn.linestreaming(series)
print(series)
#x['location']['name'], 

Lightning initialized
Connected to server at http://localhost:3000
[[-0.18667612]
 [-0.43275402]
 [-1.40955176]
 [-0.82131372]]


#### Init StramingContext and KafkaDirectStream

In [14]:
sc = SparkContext(appName="ExamenFinal")
ssc = StreamingContext(sc, 2)
ssc.checkpoint('/tmp')

Py4JError: ignored

In [12]:
topic_list = ["a1cp94e3-weather"]
brokers = {'metadata.broker.list': 'localhost:9092'}

kafkaParams_dict = {
    'metadata.broker.list': 'tricycle-01.srvs.cloudkafka.com:9094,tricycle-02.srvs.cloudkafka.com:9094,tricycle-03.srvs.cloudkafka.com:9094',
    'sasl.mechanism': 'SCRAM-SHA-256',
    'security.protocol': 'SASL_SSL',
    'sasl.username': 'a1cp94e3',
    'sasl.password': 'W8vGwbGkvoL1_ubC-SF4BhpETpVGTpyc',
    'group.id': 'a1cp94e3-consumer',
    'session.timeout.ms': 6000,
    'default.topic.config': {'auto.offset.reset': 'smallest'},
    'ssl.ca.location': 'cloudkarafka.ca'
}

#kafkaStream = KafkaUtils.createDirectStream(ssc, topic_list, brokers)
kafkaStream = KafkaUtils.createDirectStream(ssc, topics = topic_list, kafkaParams = kafkaParams_dict)

NameError: ignored

#### Streaming

In [ ]:
jsons = kafkaStream.map(lambda x: json.loads(x[1]))

jsons1 = jsons.map(lambda x: (x['current']['weather_descriptions'][0], x['current']['temperature'])).groupByKey().reduceByKey(lambda x,y: x+y).map(lambda x: (x[0], len(x[1])))
jsons1.pprint()

def measures(data):
    return data[1]
    
def sendViz(rdd):
    ll = rdd.foreach(measures)
    arr = ll.collect()
    arr.pprint()
    viz.append(np.array(arr))
    return arr

#plot = jsons.map(lambda x: measures(x, cols))
#plot.foreachRDD(lambda rdd: rdd.foreach(sendRecord))

ssc.start()
ssc.awaitTermination(700)

Py4JJavaError: An error occurred while calling o24.awaitTerminationOrTimeout.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/sarah/opt/miniconda3/lib/python3.7/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/sarah/opt/miniconda3/lib/python3.7/site-packages/pyspark/streaming/dstream.py", line 161, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "<ipython-input-5-7bd1039beef2>", line 16, in <lambda>
    jsons1.foreachRDD(lambda rdd: sendViz(rdd))
  File "<ipython-input-5-7bd1039beef2>", line 11, in sendViz
    arr = ll.collect()
AttributeError: 'NoneType' object has no attribute 'collect'

	at org.apache.spark.streaming.api.python.TransformFunction.callPythonTransformFunction(PythonDStream.scala:95)
	at org.apache.spark.streaming.api.python.TransformFunction.apply(PythonDStream.scala:78)
	at org.apache.spark.streaming.api.python.PythonDStream$$anonfun$

In [ ]:
ssc.stop(stopSparkContext=True)
sc.stop()

In [ ]:
conf = {
    'bootstrap.servers': 'tricycle-01.srvs.cloudkafka.com:9094,tricycle-02.srvs.cloudkafka.com:9094,tricycle-03.srvs.cloudkafka.com:9094',
    'group_id': 'a1cp94e3-consumer',
    'request_timeout_ms': 6000,
    #'default.topic.config': {'auto.offset.reset': 'smallest'},
    'security_protocol': 'SASL_SSL',
    'sasl_mechanism': 'SCRAM-SHA-256',
    'sasl_plain_username': 'a1cp94e3',
    'sasl_plain_password': 'W8vGwbGkvoL1_ubC-SF4BhpETpVGTpyc',
    'ssl_cafile': 'cloudkarafka.ca'
}